<a href="https://colab.research.google.com/github/samgregson/GHPT-colab-experiments/blob/main/Grasshopper_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
install openai and setup API key

This notebook works both in a Colab environment and on local machine

Colab:
- API key must be saved in Colab sectrets as OPENAI_API_KEY

Local:
- API key must be defined in the .env file (refer to example.env)

In [16]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
IN_COLAB

False

In [17]:
if IN_COLAB:
    !pip install openai
    !pip install requests

if IN_COLAB:
    import os
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

### load files into Colab and install package

NOTE: you may need to refresh your Colab files directory to see changes

In [18]:
if IN_COLAB:
    # remove the existing directory
    import shutil
    shutil.rmtree('/content/GHPT-experiments/', ignore_errors=True)
    !git clone "https://github.com/samgregson/GHPT-experiments"
    !pip install -e /content/GHPT-experiments/
    # add the modules to the search path
    import site
    site.main()

# OpenAI

In [19]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai

client = OpenAI()

# Custom wrap for VSCode, needs to be the first wrap!
if not IN_COLAB:
    from patch_openai.patch_openai import patch_openai
    client = patch_openai(client)

# Wrap the OpenAI client with LangSmith
client = wrap_openai(client)

# GHPT

### Import prompts

In [20]:
from prompts.generate_script import system_prompt, prompt_template

# replace all { with {{ and } with }} to escape the curly braces
prompt_template = prompt_template.replace("{","{{").replace("}","}}").replace("{{QUESTION}}","{QUESTION}")

In [21]:
# example use:
prompt_template.format(QUESTION="create a sphere")

'\n// Question : How do I add two numbers inside of Grasshopper?\n// Reasoning: To add two numbers we need to Add. There is an Addition component that performs this function. We need to create two numbers the user can edit, we can use the Number Slider for both numbers. And then we can Connections all of the components together\n// JSON:\n{\n\t"Advice": "Make sure to set the number sliders to the correct value",\n\t"Additions": [\n\t\t{\n\t\t\t"Name": "Number Slider",\n\t\t\t"Id": 1,\n\t\t\t"value": "0..25..100"\n\t\t},\n\t\t{\n\t\t\t"Name": "Number Slider",\n\t\t\t"Id": 2,\n\t\t\t"value": "-50..25..100"\n\t\t},\n\t\t{\n\t\t\t"Name": "Addition",\n\t\t\t"Id": 3\n\t\t}\n\t],\n\t"Connections": [\n\t\t{\n\t\t\t"To": {\n\t\t\t\t"Id": 3,\n\t\t\t\t"ParameterName": "A"\n\t\t\t},\n\t\t\t"From": {\n\t\t\t\t"Id": 1,\n\t\t\t\t"ParameterName": "number"\n\t\t\t}\n\t\t},\n\t\t{\n\t\t\t"To": {\n\t\t\t\t"Id": 3,\n\t\t\t\t"ParameterName": "B"\n\t\t\t},\n\t\t\t"From": {\n\t\t\t\t"Id": 2,\n\t\t\t\t"Parame

In [22]:
system_prompt

"\nYou are a Grasshopper Expert and are going to help create Grasshopper Definitions.\nKeep the answers short and concise.\nMake sure you create and connect a component for every non-optional input\nIf you're not sure about the answer, but think there's additional information that could help you, please ask for that information.\n\nAlways use the given format, avoid any devitation.\n"

# Instructor

In [23]:
if IN_COLAB:
    !pip install -U instructor
    !pip install anthropic

In [24]:
import instructor

client_instructor = instructor.patch(client)

In [25]:
from langsmith import traceable
from models.models import GrasshopperScriptModel

# create the openai api call function
@traceable
def call_openai_instructor(prompt: str, system_prompt: str = "", model: str = "gpt-3.5-turbo-1106", temperature: float = 0):
    completion = client_instructor.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        response_model=GrasshopperScriptModel,
        max_retries = 3
    )

    return completion

In [26]:
model = "gpt-3.5-turbo-1106"
# model = "gpt-4-1106-preview"
response = call_openai_instructor(prompt='Create a cone with a base radius of 5 and a height of 10', system_prompt=system_prompt)

InstructorRetryException: 1 validation error for ChatCompletion
  Invalid JSON: expected ident at line 1 column 3 [type=json_invalid, input_value=b'Internal Server Error', input_type=bytes]
    For further information visit https://errors.pydantic.dev/2.7/v/json_invalid

In [ ]:
print(response.json())

NameError: name 'response' is not defined